In [1]:
#Importing all modules
import sys
import time
import os

import pandas as pd
import math
import numpy as np
import random

from loguru import logger
from pathlib import Path

from Naive.node import Node

In [2]:
#time_series_data = pd.read_csv('Dataset/Sales_Transaction_Dataset_Weekly_Final.csv')
time_series_data = pd.read_csv('Dataset/cgm.csv')
time_series_data


,PID,Rec1,Rec2,Rec3,Rec4,Rec5,Rec6,Rec7,Rec8,Rec9,...,Rec11,Rec12,Rec13,Rec14,Rec15,Rec16,Rec17,Rec18,Rec19,Rec20
0,1,194,204,201,204,200,196,192,191,186,...,162,162,156,150,147,145,142,137,127,126
1,2,163,168,163,180,203,226,244,255,285,...,302,315,327,336,334,323,321,321,318,308
2,3,52,52,48,52,55,62,67,58,66,...,88,92,98,99,115,136,162,183,198,219
3,4,112,107,108,118,129,134,137,143,152,...,174,179,183,188,176,167,173,160,145,132
4,5,170,177,181,189,199,211,222,230,232,...,239,243,243,242,216,215,218,219,219,219
5,6,188,190,192,185,180,174,164,151,135,...,122,118,115,112,107,102,100,101,92,87
6,7,109,107,111,114,114,114,116,116,120,...,121,122,122,122,123,123,124,123,120,120
7,8,153,162,171,162,163,158,150,140,133,...,122,120,125,131,138,145,139,142,139,131
8,9,115,117,116,116,119,123,125,123,120,...,111,108,106,100,103,102,99,98,104,103
9,10,103,105,101,100,105,110,108,115,118,...,109,106,103,104,101,101,101,102,104,108


In [3]:
columns = list(time_series_data.columns)
time_series_index = columns.pop(0)  # remove product code

time_series_dict = dict()

for index, row in time_series_data.iterrows():
    time_series_dict[row[time_series_index]] = list(row[columns])



In [4]:
def create_tree_phase(time_series_data, P_value, paa_value,max_level):
    
    good_leaf_nodes = list()
    bad_leaf_nodes = list()

    print("Create-tree phase: start node splitting")
    node = Node(level=1, group=time_series_data, paa_value=5)
    node.start_splitting(P_value, max_level, good_leaf_nodes, bad_leaf_nodes) # using naive method node splitting 
    
    return good_leaf_nodes, bad_leaf_nodes
    

In [5]:
def recycle_bad_leaves_phase(p_value,good_leaf_nodes, bad_leaf_nodes,paa_value):
    
    suppressed_nodes = list()
    bad_leaves_node_dict = dict()
        
    for node in bad_leaf_nodes:
        if node.level in bad_leaves_node_dict.keys():
            bad_leaves_node_dict[node.level].append(node)
        else:
            bad_leaves_node_dict[node.level] = [node]

    bad_leaf_nodes_size = sum([node.size for node in bad_leaf_nodes])
    
        
    if bad_leaf_nodes_size >= p_value:
        
        current_level = max(bad_leaves_node_dict.keys())
        
        while bad_leaf_nodes_size >= p_value:
            
            if current_level in bad_leaves_node_dict.keys():
                leave_merge_dict = dict()
                keys_remove_list = list()
                merge = False
                
                for current_level_node in bad_leaves_node_dict[current_level]:
                    pattern_rep_node = current_level_node.pattern_representation
                    if pattern_rep_node in leave_merge_dict.keys():
                        merge = True
                        leave_merge_dict[pattern_rep_node].append(current_level_node)
                        if pattern_rep_node in keys_remove_list:
                            keys_remove_list.remove(pattern_rep_node)
                        else:
                            leave_merge_dict[pattern_rep_node] = [current_level_node]
                            keys_remove_list.append(pattern_rep_node)
                    
                    if merge:
                        for k in keys_remove_list:
                            del leave_merge_dict[k]

                        for pr, node_list in leave_merge_dict.items():
                            group = dict()
                            for node in node_list:
                                bad_leaves_node_dict[current_level].remove(node)
                                group.update(node.group)
                            if current_level > 1:
                                level = current_level
                            else:
                                level = 1
                            leaf_merge = Node(level=level, pattern_representation=pr,
                                group=group, paa_value=paa_value)

                            if leaf_merge.size >= p_value:
                                leaf_merge.label = "good-leaf"
                                good_leaf_nodes.append(leaf_merge)
                                bad_leaf_nodes_size -= leaf_merge.size
                            else: 
                                leaf_merge.label = "bad-leaf"
                                bad_leaves_node_dict[current_level].append(leaf_merge)

                new_level = current_level-1
                for node in bad_leaves_node_dict[current_level]:
                    if new_level > 1:
                        values_group = list(node.group.values())
                        data = np.array(values_group[0])
                        data_znorm = znorm(data)
                        data_paa = paa(data_znorm, paa_value)
                        pr = ts_to_string(data_paa, cuts_for_asize(new_level))
                    else:
                        pr = "a"*paa_value
                    node.level = new_level
                    node.pattern_representation = pr

                if current_level > 0:
                    if new_level not in bad_leaves_node_dict.keys():
                        bad_leaves_node_dict[new_level] = bad_leaves_node_dict.pop(current_level)
                    else:
                        bad_leaves_node_dict[new_level] = bad_leaves_node_dict[new_level] + bad_leaves_node_dict.pop(current_level) 
                    current_level -= 1
                else:
                    break 

        
        remaining_bad_leaf_nodes = list(bad_leaves_node_dict.values())[0]
        for node in remaining_bad_leaf_nodes:
            suppressed_nodes.append(node)
        print (suppressed_nodes)

    return suppressed_nodes

In [6]:

def top_down_greedy_clustering(T, size, T_clustered,T_structure, label='o', T_max_vals=None, T_min_vals=None):
    
    if len(T) < 2*size:
        T_clustered.append(T)
        T_structure.append(label)
        return

    ids = list(T.keys())

    # 1. Initialize groups via a NCP maximization-based heuristic
    group_u = dict()
    group_v = dict()

    seed = ids[random.randint(0, len(ids) - 1)] # Draw a random row Id
    group_u[seed] = T[seed]

    old = seed # Last visited record

    # to avoid this row to end up in two different groups
    del T[seed]
    ids.remove(seed)


    rounds = 6 if len(T) >= 6 else len(T)

    for rnd in range(rounds):
        if rnd % 2 == 0:
            source = group_u
            target = group_v
        else:
            source = group_v
            target = group_u
            
        r = find_tuple_with_max_vl(source[old], T, old)

        target[r] = T[r]
        old = r

        # Update data structures
        del T[r]
        ids.remove(r)

    # 1.b Assign each record to the group with lower NCP
    random.shuffle(ids) # Shuffle leftover Ids

    for i in ids:
        row = T[i]

        # Copy values to check what would happen
        # if row was added to either one separately
        group_u_vals = list(group_u.values())
        group_v_vals = list(group_v.values())

        group_u_vals.append(row)
        group_v_vals.append(row)
        
        metric_u = instant_value_loss(group_u_vals)
        metric_v = instant_value_loss(group_v_vals)

        if metric_v < metric_u:
            group_v[i] = row
            del group_u_vals[-1]
        else:
            group_u[i] = row
            del group_v_vals[-1]

        del T[i]

    # 2. Iterate recursively, or store groups if base case
    if len(group_u) >= size:
        top_down_greedy_clustering(group_u, size, T_clustered, T_structure, label + 'a', T_max_vals, T_min_vals) 
        T_clustered.append(group_u)
        T_structure.append(label + 'a')

    if len(group_v) >= size:
        top_down_greedy_clustering(group_v, size, T_clustered, T_structure, label + 'b', T_max_vals, T_min_vals) 
    else:
        T_clustered.append(group_v)
        T_structure.append(label + 'b')


def find_tuple_with_max_vl(base, T, key):
    max_vl = 0
    best = None

    for k in T.keys():
        if k != key:
            vl = instant_value_loss([base, T[k]])

            if vl >= max_vl: # Update the best tuple Id
                max_vl = vl
                best = k

    return best

def instant_value_loss(T, r_plus=None, r_minus=None):
  

    n = len(T[0])  # # of QI attributes in T

    if not r_plus or not r_minus:
        r_plus  = list()
        r_minus = list()


        for i in range(n): 
            r_plus_i  = 0
            r_minus_i = float('inf')

            for row in T:
                if row[i] > r_plus_i:
                    r_plus_i = row[i]

                if row[i] < r_minus_i:
                    r_minus_i = row[i]

            r_plus.append(r_plus_i) 
            r_minus.append(r_minus_i)
    
    # Compute VL(t) and then VL(T)
    vl_t = 0

    for i in range(n):
        vl_t += pow((r_plus[i] - r_minus[i]), 2) / n

    vl_T = len(T)*np.sqrt(vl_t)
    return vl_T


def top_down_greedy_clustering_postprocessing(size, T_clustered, T_structure,T_postprocessed, T_max_vals=None, 
                                              T_min_vals=None):
    idxs_merged = list()      
    groups_merged = list()    
    structure_merged = list() 

  
    for idx, bad_group in enumerate(T_clustered):
        bad_g_size = len(bad_group)
        if bad_g_size < size: # For any bad group
            bad_group_vals = list(bad_group.values())
           
            label = T_structure[idx]

            
            idx_nn = -1
            found_nn = False
            metric_nn = float('inf')

           
            for other_idx, other_label in enumerate(T_structure):
                
                if label[:-1] == other_label[:-1]: 
                    if idx == other_idx:
                        continue

                   
                    if other_idx not in idxs_merged:
                        found_nn = True
                        idx_nn = other_idx
                        break
           

            merge_with_other_group = False
            if found_nn:
                group_nn = T_clustered[idx_nn]
            elif idx_nn !=idx:
                if idx - 1 > 0:
                    idx_nn = idx - 1
                elif idx + 1 < len(T_structure) - 1:
                    idx_nn = idx + 1 
                group_nn = T_clustered[idx_nn]
                merge_with_other_group = True

            if found_nn or merge_with_other_group:
                group_merged_nn = bad_group_vals

                
                group_merged_nn = group_merged_nn  + list(group_nn.values())
                
                metric_nn = instant_value_loss(group_merged_nn)

                # Redefine group_merged_nn as dict
                group_merged_nn = dict()
                group_merged_nn.update(bad_group)
                group_merged_nn.update(group_nn)

            # Find the most appropriate large group (>= 2*size -|G|) - 2nd candidate group
            metric_large_g = float('inf')
            idx_large_g = -1

            for other_idx, other_group in enumerate(T_clustered):
               
                if len(other_group) >= 2*size - bad_g_size: 
                   
                    if other_idx not in idxs_merged:
                        group_merged_large_g = bad_group.copy()
                        group_large_g_vals = list(group_merged_large_g.values())

                       
                        for j in range(size - bad_g_size): # size - |G|
                            tmp_metric = float('inf')

                            best_record = {}
                            best_row = []

                           
                            for ridx, row in other_group.items():
                                if ridx not in group_merged_large_g.keys():
                                    
                                    metric = instant_value_loss(group_large_g_vals + [ row ])

                                    if metric < tmp_metric: # Update min metric
                                        best_record = { ridx : row }
                                        tmp_metric = metric
                                        best_row = row
            
                            group_merged_large_g.update(best_record)
                            group_large_g_vals.append(best_row)

                      
                        if tmp_metric < metric_large_g:
                            metric_large_g = tmp_metric
                            idx_large_g = other_idx

                          
                            leftover_group_large_g = { k : val for (k, val)
                                    in other_group.items()
                                    if k not in group_merged_large_g.keys() }
     
            if metric_nn < metric_large_g: 
                idxs_merged.append(idx_nn)
                groups_merged.append(group_merged_nn)
                structure_merged.append(label[:-1]) 
               
            else:
               
                idxs_merged.append(idx_large_g)
                groups_merged.append(group_merged_large_g)
                groups_merged.append(leftover_group_large_g)
              
                structure_merged.append('')

           
            idxs_merged.append(idx)

   
    T_clustered = [ group for (idx, group)
            in enumerate(T_clustered)
            if idx not in idxs_merged ]
    T_clustered += groups_merged 

    T_structure = [ label for (idx, label)
            in enumerate(T_structure)
            if idx not in idxs_merged]
    T_structure += structure_merged

    T_postprocessed += T_clustered

    # 3. Check if there are any more bad groups
    bad_groups_cnt = 0

    for group in T_clustered:
        if len(group) < size:
            bad_groups_cnt +=1

    if bad_groups_cnt > 0: # Call recursively if any left
        top_down_greedy_clustering_postprocessing(size, T_clustered, T_structure,T_postprocessed, T_max_vals, T_min_vals)
        

def group_min_value_loss(group_to_search=None, group_to_merge=dict(), index_ignored=list()):
    min_p_group = {"group" : dict(), "index" : None, "vl" : float("inf")} 
    for index, group in enumerate(group_to_search):
        if index not in index_ignored: 
            vl = instant_value_loss(list(group.values()) + list(group_to_merge.values()))
            if vl < min_p_group["vl"]:
                min_p_group["vl"] = vl
                min_p_group["group"] = group
                min_p_group["index"] = index

    return min_p_group["group"], min_p_group["index"]



In [7]:
def group_formation_phase(p_subgroups, p, k ):
    
    kgroup_list= list()
    
    P_group_list = list() 
    splitted_p_subgroups = list()
    p_subgroup_split_indexes = list()
    
    print("Start group formation phase")
    
    # Initialize P_group_list 
    for p_subgroup in p_subgroups: 
        P_group_list.append(p_subgroup)

  
    for p_subgroup_index, p_subgroup in enumerate(P_group_list): 

        if len(p_subgroup) >= 2*p:
        
            tree_clustering = list()
            temp_splitted_p_subgroup = list()

            p_subgroup_to_be_splitted = p_subgroup.copy()
            top_down_greedy_clustering( p_subgroup_to_be_splitted, p, temp_splitted_p_subgroup, tree_clustering)
 
            postprocessed_p_subgroups = list()
            top_down_greedy_clustering_postprocessing(p,temp_splitted_p_subgroup,tree_clustering,postprocessed_p_subgroups) 
                                                            
            splitted_p_subgroups += postprocessed_p_subgroups
            p_subgroup_split_indexes.append(p_subgroup_index) 
    
    P_group_list = [p_subgroup for (p_subgroup_index, p_subgroup) in enumerate(P_group_list) if p_subgroup_index not in p_subgroup_split_indexes]
    P_group_list += splitted_p_subgroups
    
    p_subgroups_k_promoted_idxs = list() 

    for p_subgroup_index, p_subgroup in enumerate(P_group_list):
        
        if len(p_subgroup) >= k:
            p_subgroups_k_promoted_idxs.append(p_subgroup_index)
            kgroup_list.append(p_subgroup)

    
    P_group_list = [p_subgroup for (p_subgroup_index, p_subgroup) in enumerate(P_group_list) if p_subgroup_index not in p_subgroups_k_promoted_idxs]

    p_subgroups_index_merged = list()
    P_group_list_size= sum([len(p_subgroup) for p_subgroup in P_group_list])

  
    while P_group_list_size>= k:
     
        Group, group_index = group_min_value_loss(group_to_search=P_group_list,index_ignored=p_subgroups_index_merged)
        p_subgroups_index_merged.append(group_index) 
        P_group_list_size-= len(Group)

        while len(Group) < k:
           
            group_min, group_min_index = group_min_value_loss(P_group_list,Group,p_subgroups_index_merged)
            p_subgroups_index_merged.append(group_min_index)
            
            Group.update(group_min) 
            P_group_list_size-= len(group_min)
       
        kgroup_list.append(Group) 


    p_subgroups_left = [p_subgroup for (p_subgroup_index, p_subgroup) in enumerate(P_group_list) if p_subgroup_index not in p_subgroups_index_merged]

    # for each remaining p-subgroup
    for p_subgroup in p_subgroups_left:
       
        k_group_remaining, k_group_remaining_idx = group_min_value_loss(kgroup_list,p_subgroup)   
        kgroup_list.pop(k_group_remaining_idx)
        
        k_group_remaining.update(p_subgroup)
        kgroup_list.append(k_group_remaining)
        
    print("End group formation phase")
    
    return kgroup_list
    
    

In [8]:
def anonymized_data(pattern_Rep_data,anonymized_data,suppressed_data):
    
    final_data_anonymized= dict()
    
    for index in range(0, len(anonymized_data)): 
        
        group = anonymized_data[index]
        
        max_value = np.amax(np.array(list(group.values())), 0)
        min_value = np.amin(np.array(list(group.values())), 0)
        
        for key in group.keys():
            
            final_data_anonymized[key] = list()
            value_row = list()
            for column_index in range(0, len(max_value)):
                value_row.append("[{}-{}]".format(min_value[column_index], max_value[column_index]))
            
            
            value_row.append(pattern_Rep_data[key]) 
            value_row.append("Group: {}".format(index))

            final_data_anonymized[key] = value_row
           
        
    for index in range(0, len(suppressed_data)):
        group = suppressed_data[index]
        for key in group.keys():
            value_row = [" - "]*len(group[key])
            value_row.append(" - ") # pattern rapresentation
            value_row.append(" - ") # group
            final_data_anonymized[key] = value_row
            
    
    your_df_from_dict=pd.DataFrame.from_dict(final_data_anonymized,orient='index')
    print (your_df_from_dict)
    
    your_df_from_dict.to_csv('Dataset/out.csv') 

In [9]:
def KAPRA(K_value, P_value, paa_value ,max_level,time_series_data):
    
    #Create tree phase
    good_leaf_nodes, bad_leaf_nodes = create_tree_phase(time_series_data, P_value, paa_value,max_level)
    print ('Good leaf nodes :' + str (len (good_leaf_nodes)) + ' Bad leaf nodes :' + str (len (bad_leaf_nodes) ) )
    
    #Recycle bad-leaves phase
    if(len(bad_leaf_nodes) > 0):
        print ('\n recyling bad leaves phases starts')
        suppressed_nodes = recycle_bad_leaves_phase(P_value, good_leaf_nodes, bad_leaf_nodes, paa_value)
    
 
    suppressed_group_list =list()
    P_group_list =list()
    
    Pattern_Representation =dict()
    
    for node in suppressed_nodes:
        suppressed_group_list.append(node.group)
    
    for node in good_leaf_nodes:
        P_group_list.append(node.group)
        pr = node.pattern_representation

        for key in node.group:
            Pattern_Representation[key] = pr
    
    
    #group formation phase
    k_group_list = group_formation_phase(P_group_list, P_value,K_value)
    #print (k_group_list)

    #Anonymize data
    anonymized_data(Pattern_Representation,k_group_list,suppressed_group_list)
    

K_value = 10
P_value = 2
paa_value = 5
max_level = 4 
time_series_data = time_series_dict

KAPRA(K_value, P_value, paa_value,max_level, time_series_data)

2022-04-22 17:46:51.303 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-22 17:46:51.306 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-22 17:46:51.307 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 1 as bad-leaf and split only tg_nodes 4
2022-04-22 17:46:51.309 | INFO     | Naive.node:start_splitting:51 - Maximize-level, size:3, p_value:2 == good-leaf
2022-04-22 17:46:51.315 | INFO     | Naive.node:maximize_level_node:234 - Can't split again, max level already reached
2022-04-22 17:46:51.321 | INFO     | Naive.node:start_splitting:89 - N can be split
2022-04-22 17:46:51.323 | INFO     | Naive.node:start_splitting:90 - Compute tentative good nodes and tentative bad nodes
2022-04-22 17:46:51.325 | INFO     | Naive.node:start_splitting:152 - Label all tb_node 0 as bad-leaf and split only tg_nodes 1
2022-04-22 17:46:51.328 | INFO     | Naive.node:start_splitting:51 - Maximize-level, size:3, p_va

Create-tree phase: start node splitting
Good leaf nodes :4 Bad leaf nodes :1

 recyling bad leaves phases starts
Start group formation phase
End group formation phase
          0         1         2         3         4         5         6   \
4   [52-240]  [52-249]  [48-253]  [52-256]  [55-261]  [62-262]  [67-259]   
7   [52-240]  [52-249]  [48-253]  [52-256]  [55-261]  [62-262]  [67-259]   
1   [52-240]  [52-249]  [48-253]  [52-256]  [55-261]  [62-262]  [67-259]   
6   [52-240]  [52-249]  [48-253]  [52-256]  [55-261]  [62-262]  [67-259]   
8   [52-240]  [52-249]  [48-253]  [52-256]  [55-261]  [62-262]  [67-259]   
3   [52-240]  [52-249]  [48-253]  [52-256]  [55-261]  [62-262]  [67-259]   
12  [52-240]  [52-249]  [48-253]  [52-256]  [55-261]  [62-262]  [67-259]   
13  [52-240]  [52-249]  [48-253]  [52-256]  [55-261]  [62-262]  [67-259]   
9   [52-240]  [52-249]  [48-253]  [52-256]  [55-261]  [62-262]  [67-259]   
11  [52-240]  [52-249]  [48-253]  [52-256]  [55-261]  [62-262]  [67-259] 

PermissionError: [Errno 13] Permission denied: 'Dataset/out.csv'

In [ ]:
#Computing Instant value loss and pattern loss 

